In [3]:
#step1：导入库并加载数据
import pandas as pd
import numpy as np
from scipy import stats

# 加载 Excel 数据
file_path = r"D:\360MoveData\Users\86185\Desktop\daily return.xlsx"
df = pd.read_excel(file_path)

# 检查关键列是否存在
required_cols = ["交易日期", "调整后价格", "每日收益率(%)", "累计收益率(%)"]
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    raise KeyError(f"表格缺少必要列：{missing_cols}")

df.head(3)  # 预览数据

,交易日期,合约代码,调整后价格,每日收益率(%),合约换月标记,累计收益率(%)
0,2015-04-16,IH1505,3238.6,0.000000,0,NaN
1,2015-04-17,IH1505,3295.8,14.718294,0,14.718294
2,2015-04-17,IH1505,3295.8,0.000000,0,14.718294


In [4]:
#step2：计算 Years of data
# 提取年份并去重计数
unique_years = df["交易日期"].dt.year.nunique()
print(f"Years of data: {unique_years}")
 

Years of data: 11


In [5]:
#step3: 计算Mean annual return
total_return = (df['每日收益率(%)'] / 100).sum()

# 计算平均年化收益率（总收益率/总年数）
mean_annual_return = (total_return / unique_years) * 100  # 转回百分比

print(f"Mean annual return (总收益率/总年数): {mean_annual_return:.2f}%")

Mean annual return (总收益率/总年数): -4.03%


In [22]:
import pandas as pd

# 使用 pd.read_excel 读取 Excel 文件
df = pd.read_excel(r"D:\360MoveData\Users\86185\Desktop\daily return.xlsx")  

# 过滤掉每日收益率(%)列中值为 0 的行
filtered_df = df[df['每日收益率(%)'] != 0].copy()

# 后续计算逻辑...
filtered_df['累计净值'] = (1 + filtered_df['每日收益率(%)'] / 100).cumprod()
filtered_df['峰值'] = filtered_df['累计净值'].cummax()
filtered_df['回撤率'] = (filtered_df['峰值'] - filtered_df['累计净值']) / filtered_df['峰值']
average_drawdown = filtered_df['回撤率'].mean() * 100
print("过滤后数据的平均回撤率为：", average_drawdown)

过滤后数据的平均回撤率为： 98.74551479892565


In [8]:
#step6：计算 Sharpe ratio
RISK_FREE_RATE = 0  # 假设无风险利率为 0

# 日度SR（小数计算）
daily_sharpe = ((df["每日收益率(%)"] / 100).mean() - RISK_FREE_RATE) / (df["每日收益率(%)"] / 100).std()  

# 年化夏普比率（×√256）
annualised_sharpe = daily_sharpe * np.sqrt(256)

print(f"Sharpe ratio: {annualised_sharpe:.2f}")

Sharpe ratio: -0.01


In [9]:
#step7：计算Skew
# 月度收益率（先按月份求和）
df["month"] = df["交易日期"].dt.to_period("M")
monthly_returns = df.groupby("month")["每日收益率(%)"].sum()

# 计算偏度
skew = stats.skew(monthly_returns)
print(f"Skew: {skew:.2f}")

Skew: 2.42


In [14]:
import pandas as pd

daily_return_pct = df["每日收益率(%)"]  

# 过滤掉每日收益率(%)列中值为0的行
filtered_daily_return_pct = daily_return_pct[daily_return_pct != 0].dropna()  

# 查看过滤后的收益率分布（分位数）
print("过滤后的收益率分布（分位数）：")
print(filtered_daily_return_pct.quantile([0.01, 0.1, 0.25, 0.3, 0.5, 0.7, 0.9, 0.99]))

过滤后的收益率分布（分位数）：
0.01   -75.335605
0.10   -27.753442
0.25   -11.534588
0.30    -8.684385
0.50    -0.568440
0.70     7.685363
0.90    26.965632
0.99    80.322899
Name: 每日收益率(%), dtype: float64


In [16]:
#step8：计算 Lower tail
# 每日收益率（小数）
daily_return_pct = df["每日收益率(%)"] / 100  

# 1% 分位数与 30% 分位数（小数）
p1 = filtered_daily_return_pct.quantile(0.01)
p30 = filtered_daily_return_pct.quantile(0.30)

# Lower tail
lower_tail = (p1 / p30) / 4.43
print(f"Lower tail: {lower_tail:.2f}")

Lower tail: 1.96


In [17]:
#step8：计算 Upper tail
# 每日收益率（小数）
daily_return_pct = df["每日收益率(%)"] / 100  

# 99% 分位数与 70% 分位数（小数）
p99 = filtered_daily_return_pct.quantile(0.99)
p70 = filtered_daily_return_pct.quantile(0.70)

# Upper tail
upper_tail = (p99 / p70) / 4.43
print(f"Lower tail: {upper_tail:.2f}")

Lower tail: 2.36


In [23]:
import pandas as pd

# 整理指标数据
metrics = ["Years of data", "Mean annual return", "Average drawdown", 
           "Annualised standard deviation", "Sharpe ratio", "Skew", "Lower tail", "Upper tail"]
values = [
    unique_years, 
    f"{mean_annual_return:.2f}%", 
    f"{average_drawdown:.2f}%", 
    f"{annualised_std:.2f}%", 
    f"{annualised_sharpe:.2f}", 
    f"{skew:.2f}", 
    f"{lower_tail:.2f}",
    f"{upper_tail:.2f}"
]

# 创建多层索引，让策略和标的在同一行，指标和对应值在下方行
multi_index = pd.MultiIndex.from_tuples(
    [("Strategy: Buy and hold, single contract", "SSE 50 Index Future")] +  # 第一行：策略和标的分别在两列
    list(zip(metrics, values))  # 后续行：指标和对应值
)

# 构建新表格，列名设为空
summary_with_header = pd.DataFrame(
    index=multi_index,
    columns=[""]
)
# 填充空字符串
summary_with_header.loc[:, ""] = ""

# 显示表格
summary_with_header

,,
"Strategy: Buy and hold, single contract",SSE 50 Index Future,
Years of data,11,
Mean annual return,-4.03%,
Average drawdown,98.75%,
Annualised standard deviation,274.63%,
Sharpe ratio,-0.01,
Skew,2.42,
Lower tail,1.96,
Upper tail,2.36,
